J VS P CLASSIFICATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "J_vs_P_data.csv")
df = df.dropna(how='any',axis=0)
df.sample(10)

,label,text
12930,0,meant find home ca loved anyone still chasing ...
30304,0,'re welcome understand nobody perfect matters ...
196534,1,seems like enfj thing cough mlk cough maybe in...
134845,1,wow infj males rare feel bad
178021,1,thank much gifford expecting personal reach he...
97383,0,general ripped shreds took months realization ...
29363,0,congrats
26676,0,probably pretty good feeding tongue
197007,1,think idea behind sometimes people afraid ones...
84565,0,hey always awesome see someone around type lif...


In [3]:
DOWNSAMPLE = 30000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

0    15006
1    14994
Name: label, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [5]:
train.head(3)

,label,text
11184,0,thoughts opinions especially people told like ...
136792,1,cant speak rest people usually put time effort...
137126,1,'re favorite person website


In [6]:
test.head(3)

,label,text
99255,0,score though conscience right place also pragm...
63525,0,act exactly normal generally fairly laid back ...
133491,1,goldie esfp poster girl apparently though type...


In [7]:
ourpur_dir = binary_data_path + "JvsP_BERT/"
train.to_csv(ourpur_dir+"J_vs_P_train_data.csv", index=False)
test.to_csv(ourpur_dir+"J_vs_P_test_data.csv", index=False)

In [8]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 212 kB 98.3 MB/s 
     |████████████████████████████████| 132 kB 88.0 MB/s 
     |████████████████████████████████| 182 kB 97.8 MB/s 
     |████████████████████████████████| 127 kB 94.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [9]:
data_files = {"train": ourpur_dir+"J_vs_P_train_data.csv", "test": ourpur_dir+"J_vs_P_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')



Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e11f92453108093d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4500
    })
})

In [11]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 71.8 MB/s 


In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [14]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

4500

In [15]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/25500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [16]:
tokenized_train[0]

{'label': 0,
 'text': 'gon na lie hella great chipper grunge',
 'input_ids': [101,
  2175,
  2078,
  6583,
  4682,
  3109,
  2050,
  2307,
  9090,
  4842,
  24665,
  5575,
  2063,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [19]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [20]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7970
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.687100
1000,0.675400
1500,0.673200
2000,0.646900
2500,0.637300
3000,0.632400
3500,0.566200
4000,0.515200
4500,0.511200
5000,0.462500


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/JvsP_BERT/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/D

TrainOutput(global_step=7970, training_loss=0.5023242122406044, metrics={'train_runtime': 393.7834, 'train_samples_per_second': 323.782, 'train_steps_per_second': 20.24, 'total_flos': 962420152002576.0, 'train_loss': 0.5023242122406044, 'epoch': 5.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 16


<ipython-input-19-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 1.0358930826187134,
 'eval_accuracy': 0.6051111111111112,
 'eval_f1': 0.6018373291507954,
 'eval_runtime': 3.1829,
 'eval_samples_per_second': 1413.795,
 'eval_steps_per_second': 88.598,
 'epoch': 5.0}